In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('datasetfinal.csv')
data

,seller_id,product_name,product_id,buyer_id,product_rating,product_price
0,457,40 Set Menu Sehari-hari Hits di Instagram ala ...,30070,1504,4.9,142800
1,457,7 Hari Belajar Drone Photography (Edisi Revisi),30079,1435,5.0,102000
2,457,Akasha : City Hunter - Complete Edition 03,30121,1560,4.9,49300
3,457,AKASHA : FATE/APOCRYPHA 02,30163,271,4.8,38250
4,457,AKASHA : RECORD OF RAGNAROK 03,30219,6,5.0,38250
...,...,...,...,...,...,...
49995,105,Samyang V-AF 24mm T1.9 Lens for Sony FE Samyan...,23658,47,5.0,8979000
49996,135,Godox S60 LED Focusing 3-Light Kit,29682,132,5.0,18849000
49997,134,Memory 128GB SF-M Tough Series UHS-II SDXC Mem...,28802,250,5.0,1599000
49998,4,Insta360 One X3 Sticky Lensguard,20253,248,5.0,539000


In [4]:
import pandas as pd

def preprocess_data(data):
    # Drop baris dengan product_rating == 'No Rating'
    df = data[data['product_rating'] != 'No Rating']

    # Konversi ke float
    df['product_rating'] = df['product_rating'].astype(float)

    return df

In [5]:
data = preprocess_data(data)
data

<ipython-input-4-2209759585>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_rating'] = df['product_rating'].astype(float)


,seller_id,product_name,product_id,buyer_id,product_rating,product_price
0,457,40 Set Menu Sehari-hari Hits di Instagram ala ...,30070,1504,4.9,142800
1,457,7 Hari Belajar Drone Photography (Edisi Revisi),30079,1435,5.0,102000
2,457,Akasha : City Hunter - Complete Edition 03,30121,1560,4.9,49300
3,457,AKASHA : FATE/APOCRYPHA 02,30163,271,4.8,38250
4,457,AKASHA : RECORD OF RAGNAROK 03,30219,6,5.0,38250
...,...,...,...,...,...,...
49995,105,Samyang V-AF 24mm T1.9 Lens for Sony FE Samyan...,23658,47,5.0,8979000
49996,135,Godox S60 LED Focusing 3-Light Kit,29682,132,5.0,18849000
49997,134,Memory 128GB SF-M Tough Series UHS-II SDXC Mem...,28802,250,5.0,1599000
49998,4,Insta360 One X3 Sticky Lensguard,20253,248,5.0,539000


In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

def create_user_embedding_model(data, n_latent_factors=50):
    # Label encoding untuk user dan product
    user_encoder = LabelEncoder()
    product_encoder = LabelEncoder()

    data['user'] = user_encoder.fit_transform(data['buyer_id'])
    data['product'] = product_encoder.fit_transform(data['product_id'])

    n_users = data['user'].nunique()
    n_products = data['product'].nunique()

    # Input layers
    user_input = tf.keras.Input(shape=(1,), name='user_input')
    product_input = tf.keras.Input(shape=(1,), name='product_input')

    # Embedding layers
    user_embedding = tf.keras.layers.Embedding(input_dim=n_users, output_dim=n_latent_factors, name='user_embedding')(user_input)
    product_embedding = tf.keras.layers.Embedding(input_dim=n_products, output_dim=n_latent_factors, name='product_embedding')(product_input)

    # Flatten embeddings
    user_vec = tf.keras.layers.Flatten()(user_embedding)
    product_vec = tf.keras.layers.Flatten()(product_embedding)

    # Model (output hanya user vector, sesuai kode awal)
    model = tf.keras.Model(inputs=user_input, outputs=user_vec)

    # Return model
    return model

In [7]:
model = create_user_embedding_model(data)

<ipython-input-6-483179597>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user'] = user_encoder.fit_transform(data['buyer_id'])
<ipython-input-6-483179597>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['product'] = product_encoder.fit_transform(data['product_id'])


In [8]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)         │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ user_embedding (Embedding)      │ (None, 1, 50)          │       125,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 125,000 (488.28 KB)

 Trainable params: 125,000 (488.28 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.save("model.h5")

# Install tensorflowjs
!pip install tensorflowjs

# Convert model.h5 to model
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model